### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Susceptibility_to_Infections/GSE150133'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Assuming sample_characteristics is already defined/imported prior to this code block
from previous_step import sample_characteristics

# Check for the availability of gene expression data based on Background Information
is_gene_available = True

# Determine the availability of 'Susceptibility_to_Infections', 'age', and 'gender' data
# and set the corresponding keys appropriately.
trait_row = 4 if len(set([val.split(': ')[1] for val in sample_characteristics[4]])) > 1 else None
age_row = 1 if len(set([val.split(': ')[1] for val in sample_characteristics[1]])) > 1 else None
gender_row = 2 if len(set([val.split(': ')[1] for val in sample_characteristics[2]])) > 1 else None

# Define conversion functions for the identified variables
def convert_trait(value):
    split_val = value.split(': ')[1]
    if split_val == 'Yes':
        return 1
    elif split_val == 'No':
        return 0
    else:
        return None

def convert_age(value):
    try:
        return float(value.split(': ')[1])
    except ValueError:
        return None

def convert_gender(value):
    split_val = value.split(': ')[1]
    if split_val == 'Male':
        return 1
    elif split_val == 'Female':
        return 0
    else:
        return None

# Save cohort information
save_cohort_info('GSE150133', './preprocessed/Susceptibility_to_Infections/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical feature extraction if clinical data is available
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Susceptibility_to_Infections', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Susceptibility_to_Infections/trait_data/GSE150133.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
